In [14]:
import requests
from datetime import datetime, timedelta
import pandas as pd

ORION_URL = "http://localhost:1026"
NGSI_LD_ENDPOINT = f"{ORION_URL}/ngsi-ld/v1/entities"

print(f"Orion-LD URL: {ORION_URL}")

Orion-LD URL: http://localhost:1026


In [15]:
response = requests.get(f"{ORION_URL}/version")
print(response.json())

{'orionld version': '1.4.0', 'orion version': '1.15.0-next', 'uptime': '0 d, 0 h, 5 m, 54 s', 'git_hash': 'nogitversion', 'compile_time': 'Sat Aug 26 06:19:09 UTC 2023', 'compiled_by': 'root', 'compiled_in': '', 'release_date': 'Sat Aug 26 06:19:09 UTC 2023', 'doc': 'https://fiware-orion.readthedocs.org/en/master/'}


In [16]:
df = pd.read_excel('/home/cc/DigiEV/rawData/20240530_20250530_recharge_detail_records.xlsx')
df = df.iloc[:, 1:] # 读取后删除第一列
df = df[df['location_name'] == 'PS 030-1 - IPP - Drahi - X']
df.head() 

,transaction_id,id_ocpp,location_name,id_pl,start_volume,end_volume,total_volume,startdate,enddate,duration,authorize,rfid,book_id,auth_method
0,512840826,ParisSaclay30-1,PS 030-1 - IPP - Drahi - X,1,2275.265,2293.518,18.253,2025-05-30T12:04:57Z,2025-05-30T13:38:43Z,5626,RemoteApp,1,NaN,RFID
1,703497342,ParisSaclay30-1,PS 030-1 - IPP - Drahi - X,1,2274.677,2275.181,0.504,2025-05-30T05:42:28Z,2025-05-30T05:44:44Z,136,8E47015B,8E47015B,NaN,RFID
2,412512340,ParisSaclay30-1,PS 030-1 - IPP - Drahi - X,1,2272.276,2274.677,2.401,2025-05-29T22:34:44Z,2025-05-29T22:45:24Z,640,8E47015B,8E47015B,NaN,RFID
5,852888750,ParisSaclay30-1,PS 030-1 - IPP - Drahi - X,1,2264.202,2272.276,8.074,2025-05-29T10:38:30Z,2025-05-29T11:27:56Z,2966,8E47015B,8E47015B,NaN,RFID
7,718583992,ParisSaclay30-1,PS 030-1 - IPP - Drahi - X,1,2263.346,2264.202,0.856,2025-05-28T17:00:49Z,2025-05-29T06:05:31Z,47082,8E47015B,8E47015B,NaN,RFID


In [17]:
def create_charging_session_entities(df):
    entities = []
    
    for index, row in df.iterrows():
        start_time = pd.to_datetime(row['startdate'])
        end_time = pd.to_datetime(row['enddate'])
        
        # Time Europe/Paris
        if start_time.tz is None:
            start_time = start_time.tz_localize('Europe/Paris')
        if end_time.tz is None:
            end_time = end_time.tz_localize('Europe/Paris')
        
        entity = {
            "id": f"urn:ngsi-ld:ChargingSession:session-{row['transaction_id']}",
            "type": "ChargingSession",
            
            "refChargingPoint": {
                "type": "Relationship",
                "object": "urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01"
            },
            
            "sessionId": {
                "type": "Property",
                "value": str(row['transaction_id'])
            },
            
            "sessionStatus": {
                "type": "Property",
                "value": "ended"
            },
            
            "sessionType": {
                "type": "Property",
                "value": "AC-Level2"
            },
            
            "sessionStartTime": {
                "type": "Property",
                "value": {
                    "@type": "DateTime",
                    "@value": start_time.isoformat()
                },
                "observedAt": start_time.isoformat()
            },
                      
            "sessionEndTime": {
                "type": "Property",
                "value": {
                    "@type": "DateTime",
                    "@value": end_time.isoformat()
                },
                "observedAt": end_time.isoformat()
            },
            
            "sessionDuration": {
                "type": "Property",
                "value": float(row['duration']) / 60,
                "units": "MIN",
                "observedAt": end_time.isoformat()
            },
            
            "energyConsumed": {
                "type": "Property",
                "value": float(row['total_volume']),
                "units": "kWh",
                "observedAt": end_time.isoformat()               
            },
            
            "userId": {
                "type": "Property",
                "value": str(row['authorize'])
            },
            
            "authenticationMethod": {
                "type": "Property",
                "value": str(row['auth_method'])
            },
            
            "authenticationId": {
                "type": "Property",
                "value": str(row['rfid'])
            },
            
            "dateCreated": {
                "type": "Property",
                "value": {
                    "@type": "DateTime",
                    "@value": start_time.isoformat()
                }
            },
            
            "lastUpdated": {
                "type": "Property",
                "value": {
                    "@type": "DateTime",
                    "@value": datetime.now().isoformat() + "Z"
                }
            }
        }
        
        entities.append(entity)
    
    return entities

In [18]:
entities = create_charging_session_entities(df)

In [19]:
def upload_to_context_broker(entities, ngsi_ld_endpoint):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/ld+json',
        'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }
    
    for entity in entities:
        response = requests.post(ngsi_ld_endpoint, json=entity, headers=headers)
        print(f"HTTP status: {response.status_code}")
        print(f"info: {response.text}")

In [20]:
upload_to_context_broker(entities, "http://localhost:1026/ngsi-ld/v1/entities")

HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201
info: 
HTTP status: 201

In [21]:
headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/ld+json',
        'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }

In [22]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingPoint"

response = requests.get(query_url, headers=headers)
print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: []


In [23]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingSession"

response = requests.get(query_url, headers=headers)
print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: [{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld', 'id': 'urn:ngsi-ld:ChargingSession:session-512840826', 'type': 'ChargingSession', 'authenticationId': {'type': 'Property', 'value': '1'}, 'authenticationMethod': {'type': 'Property', 'value': 'RFID'}, 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-05-30T12:04:57+00:00'}}, 'energyConsumed': {'type': 'Property', 'value': 18.253, 'units': {'type': 'Property', 'value': 'kWh'}, 'observedAt': '2025-05-30T13:38:43.000Z'}, 'lastUpdated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-06-27T16:12:03.968460Z'}}, 'refChargingPoint': {'type': 'Relationship', 'object': 'urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01'}, 'sessionDuration': {'type': 'Property', 'value': 93.766667, 'units': {'type': 'Property', 'value': 'MIN'}, 'observedAt': '2025-05-30T13:38:43.000Z'}, 'sessionEndTime': {'type': 'Prop

In [24]:
query_url = f"{NGSI_LD_ENDPOINT}?type=ChargingSession&q=refChargingPoint==\"urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01\""

response = requests.get(query_url, headers=headers)

print(f"HTTP status: {response.status_code}")
print(f"info: {response.json()}")

HTTP status: 200
info: [{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld', 'id': 'urn:ngsi-ld:ChargingSession:session-512840826', 'type': 'ChargingSession', 'authenticationId': {'type': 'Property', 'value': '1'}, 'authenticationMethod': {'type': 'Property', 'value': 'RFID'}, 'dateCreated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-05-30T12:04:57+00:00'}}, 'energyConsumed': {'type': 'Property', 'value': 18.253, 'units': {'type': 'Property', 'value': 'kWh'}, 'observedAt': '2025-05-30T13:38:43.000Z'}, 'lastUpdated': {'type': 'Property', 'value': {'@type': 'DateTime', '@value': '2025-06-27T16:12:03.968460Z'}}, 'refChargingPoint': {'type': 'Relationship', 'object': 'urn:ngsi-ld:ChargingPoint:ParisSaclay30-1-P-01'}, 'sessionDuration': {'type': 'Property', 'value': 93.766667, 'units': {'type': 'Property', 'value': 'MIN'}, 'observedAt': '2025-05-30T13:38:43.000Z'}, 'sessionEndTime': {'type': 'Prop